In [1]:
import pandas as pd
import os
import pickle

In [2]:
def save_dictionary(path, data):
    """
    This function saves a dictionary to a specified file path using pickle.
    
    Parameters:
    path (str): The file path where the dictionary will be saved.
    data (dict): The dictionary data to be saved.
    
    Returns:
    No return value, this function writes the dictionary to the specified file path.
    """
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    with open(path, 'wb') as file:
        pickle.dump(data, file)

In [3]:
def load_dictionary(path):
    """
    This function loads a dictionary from a specified file path using pickle.
    
    Parameters:
    path (str): The file path from which the dictionary will be loaded.
    
    Returns:
    dict: The dictionary data loaded from the specified file.
    """
    with open(path, 'rb') as file:
        loaded_data = pickle.load(file)
    return loaded_data

In [4]:
def print_latex_table(df, index=False, caption="Sample Table", label="tab:sample", float_format="%.2f"):
    """
    Prints the LaTeX table code for a given pandas DataFrame, allowing the table to fit within the page width
    and span multiple pages if necessary.

    Parameters:
    df (pd.DataFrame): The DataFrame to convert to a LaTeX table.
    index (bool): Whether to include the DataFrame index in the table. Default is False.
    caption (str): The caption for the table. Default is "Sample Table".
    label (str): The label for the table. Default is "tab:sample".
    float_format (str): Format string for floating point numbers. Default is "%.2f".
    """
    # Escape special characters in the DataFrame columns and cells
    df.columns = [escape_latex_special_chars(col) for col in df.columns]
    df = df.applymap(lambda x: escape_latex_special_chars(float_format % x) if isinstance(x, (float, int)) else escape_latex_special_chars(str(x)))

    # Calculate the number of columns and set a dynamic width
    num_columns = len(df.columns)
    col_width = 0.9 / num_columns

    # Define column format to fit all columns on one page
    col_format = '>{\\raggedright\\arraybackslash}p{' + '{:.3f}\\textwidth}}|'.format(col_width)
    col_format += '|'.join(['>{\\raggedright\\arraybackslash}p{' + '{:.3f}\\textwidth}}'.format(col_width) for _ in df.columns[1:]])

    # Start building the LaTeX table
    table = r'\begin{longtable}{|' + col_format + '|}' + "\n"
    table += r'\caption{' + caption + r'}\label{' + label + r'} \\' + "\n"
    table += r'\hline' + "\n"
    table += r'\small ' + ' & '.join(df.columns) + r' \\' + "\n"
    table += r'\hline' + "\n"
    table += r'\endfirsthead' + "\n"
    table += r'\caption[]{(continued)} \\' + "\n"
    table += r'\hline' + "\n"
    table += r'\small ' + ' & '.join(df.columns) + r' \\' + "\n"
    table += r'\hline' + "\n"
    table += r'\endhead' + "\n"
    table += r'\hline' + "\n"
    table += r'\endfoot' + "\n"
    table += r'\hline' + "\n"
    table += r'\endlastfoot' + "\n"

    # Add the rows with merged cells
    prev_labeling_method = None
    for i, row in enumerate(df.itertuples(index=index, name=None)):
        row_data = [escape_latex_special_chars(str(item)) for item in row]
        if row[0] == prev_labeling_method:
            table += r'\small ' + ' & ' + ' & '.join(row_data[1:]) + r' \\' + "\n"
        else:
            rowspan = (df.iloc[:, 0] == row[0]).sum()
            if rowspan > 1:
                table += r'\multirow{' + str(rowspan) + r'}{*}{\parbox[t]{2cm}{' + row_data[0] + r'}} & \small ' + ' & '.join(row_data[1:]) + r' \\' + "\n"
            else:
                table += r'\small ' + row_data[0] + ' & ' + ' & '.join(row_data[1:]) + r' \\' + "\n"
            prev_labeling_method = row[0]

    table += r'\end{longtable}' + "\n"

    print(table)

In [5]:
def escape_latex_special_chars(text):
    """
    Escapes special characters in a string for LaTeX.

    Parameters:
    text (str): The input string.

    Returns:
    str: The escaped string.
    """
    special_chars = {
        '&': r'\&', '%': r'\%', '$': r'\$', '#': r'\#',
        '_': r'\_', '{': r'\{', '}': r'\}', '~': r'\textasciitilde{}',
        '^': r'\textasciicircum{}', '\\': r'\textbackslash{}'
    }
    for char, escaped_char in special_chars.items():
        text = text.replace(char, escaped_char)
    return text

In [6]:
def print_latex_table_small(df, index=False, caption="Sample Table", label="tab:sample", float_format="%.2f"):
    """
    Prints the LaTeX table code for a given pandas DataFrame, allowing the table to fit within the page width
    and span multiple pages if necessary.

    Parameters:
    df (pd.DataFrame): The DataFrame to convert to a LaTeX table.
    index (bool): Whether to include the DataFrame index in the table. Default is False.
    caption (str): The caption for the table. Default is "Sample Table".
    label (str): The label for the table. Default is "tab:sample".
    float_format (str): Format string for floating point numbers. Default is "%.2f".
    """

    def escape_latex_special_chars(text):
        """
        Escapes LaTeX special characters in a given text.
        """
        special_chars = {
            '&': r'\&', '%': r'\%', '$': r'\$', '#': r'\#', '_': r'\_',
            '{': r'\{', '}': r'\}', '~': r'\textasciitilde{}',
            '^': r'\textasciicircum{}', '\\': r'\textbackslash{}'
        }
        for char, replacement in special_chars.items():
            text = text.replace(char, replacement)
        return text

    # Escape special characters in the DataFrame columns and cells
    df.columns = [escape_latex_special_chars(col) for col in df.columns]
    df = df.applymap(lambda x: escape_latex_special_chars(float_format % x) if isinstance(x, (float, int)) else escape_latex_special_chars(str(x)))

    # Calculate the number of columns and set a dynamic width
    num_columns = len(df.columns)
    
    # Set the first column width to 0.4 (to avoid line breaks in names) and distribute the remaining space to other columns
    name_col_width = 0.25
    other_col_width = (0.9 - name_col_width) / (num_columns - 1)

    # Define column format to fit all columns on one page
    col_format = 'p{' + '{:.3f}\\textwidth'.format(name_col_width) + '}|'
    col_format += '|'.join(['p{' + '{:.3f}\\textwidth'.format(other_col_width) + '}' for _ in df.columns[1:]])

    # Start building the LaTeX table
    table = r'\begin{longtable}{|' + col_format + '|}' + "\n"
    table += r'\caption{' + caption + r'}\label{' + label + r'} \\' + "\n"
    table += r'\hline' + "\n"
    table += r'\small ' + ' & '.join(df.columns) + r' \\' + "\n"
    table += r'\hline' + "\n"
    table += r'\endfirsthead' + "\n"
    table += r'\caption[]{(continued)} \\' + "\n"
    table += r'\hline' + "\n"
    table += r'\small ' + ' & '.join(df.columns) + r' \\' + "\n"
    table += r'\hline' + "\n"
    table += r'\endhead' + "\n"
    table += r'\hline' + "\n"
    table += r'\endfoot' + "\n"
    table += r'\hline' + "\n"
    table += r'\endlastfoot' + "\n"

    # Add the rows
    for row in df.itertuples(index=index, name=None):
        row_data = [escape_latex_special_chars(str(item)) for item in row]
        table += r'\small ' + ' & '.join(row_data) + r' \\' + "\n"

    table += r'\end{longtable}' + "\n"

    print(table)

In [7]:
def basic_latex_table(df):
    """
    This function takes a DataFrame and prints it as a beautifully formatted LaTeX table.
    
    Parameters:
    df (DataFrame): The DataFrame to be converted to LaTeX table.
    """
    # Format the DataFrame to round floats to 2 decimal places
    df = df.round(2)

    # Get the number of columns and their names
    num_columns = len(df.columns)
    column_names = df.columns

    # Create the column format string for LaTeX
    column_format = "|l|" + "c|" * num_columns

    # Convert DataFrame to LaTeX table with additional formatting
    latex_table = df.to_latex(
        index=True,  # Include row names (index)
        header=True,  # Include column names
        bold_rows=True,  # Make row names bold
        column_format=column_format,  # Format for columns
        escape=False,  # Do not escape LaTeX special characters
        float_format="%.2f"  # Format floats to 2 decimal places
    )

    # Clean up the LaTeX table output to remove the extra column names
    table_lines = latex_table.splitlines()
    table_body = "\n".join(table_lines[4:-2])  # Extract the body of the table

    # Construct the final LaTeX table
    latex_header = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        f"\\renewcommand{{\\arraystretch}}{{1.5}}\n"  # Increase space between rows
        f"\\begin{{tabular}}{{{column_format}}}\n"
        "\\hline\n"
        "\\textbf{Method} & " + " & ".join([f"\\textbf{{{name}}}" for name in column_names]) + " \\\\\n"
        "\\hline\n"
    )
    
    latex_footer = (
        "\\hline\n"
        "\\end{tabular}\n"
        "\\caption{Label Distribution}\n"
        "\\label{tab:label_distribution}\n"
        "\\end{table}"
    )

    # Combine header, body, and footer
    latex_table = latex_header + table_body + latex_footer
    
    print(latex_table)

In [8]:
def print_key_ml_to_latex(daily_df, weekly_df):
    """
    This function takes two DataFrames (daily and weekly) and prints a LaTeX table comparing their metrics.
    
    Parameters:
    daily_df (DataFrame): The DataFrame containing the daily metrics.
    weekly_df (DataFrame): The DataFrame containing the weekly metrics.
    
    Returns:
    None: This function prints the LaTeX table code.
    """
    # Ensure that the input dataframes have the same structure
    assert daily_df.columns.tolist() == weekly_df.columns.tolist(), "The columns of daily_df and weekly_df must match."
    assert daily_df['Labeling Method'].tolist() == weekly_df['Labeling Method'].tolist(), "The Labeling Method rows of daily_df and weekly_df must match."
    
    # Define the LaTeX table header
    header = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        "\\begin{tabular}{|l|c|c|c|c|c|c|}\n"
        "\\hline\n"
        "\\multirow{2}{*}{Labeling Method} & \\multicolumn{3}{c|}{Daily} & \\multicolumn{3}{c|}{Weekly} \\\\\n"
        "\\cline{2-7}\n"
        " & Train Acc & Test Acc & Balanced Acc & Train Acc & Test Acc & Balanced Acc \\\\\n"
        "\\hline\n"
    )
    
    # Define the LaTeX table footer
    footer = (
        "\\end{tabular}\n"
        "\\caption{Comparison of Daily and Weekly Metrics}\n"
        "\\label{tab:daily_weekly_comparison}\n"
        "\\end{table}"
    )
    
    # Generate the table body
    body = ""
    for index, row in daily_df.iterrows():
        method = row['Labeling Method']
        daily_train_acc = row['Train Accuracy']
        daily_test_acc = row['Test Accuracy']
        daily_bal_acc = row['Balanced Accuracy']
        
        weekly_row = weekly_df[weekly_df['Labeling Method'] == method]
        weekly_train_acc = weekly_row['Train Accuracy'].values[0]
        weekly_test_acc = weekly_row['Test Accuracy'].values[0]
        weekly_bal_acc = weekly_row['Balanced Accuracy'].values[0]
        
        body += (
            f"{method} & {daily_train_acc:.2f} & {daily_test_acc:.2f} & {daily_bal_acc:.2f} & "
            f"{weekly_train_acc:.2f} & {weekly_test_acc:.2f} & {weekly_bal_acc:.2f} \\\\\n"
        )
        body += "\\hline\n"
    
    # Combine the header, body, and footer
    latex_table = header + body + footer
    
    print(latex_table)

In [9]:
def print_key_ml_to_latex(daily_df, weekly_df):
    """
    This function takes two DataFrames (daily and weekly) and prints a LaTeX table comparing their metrics.
    
    Parameters:
    daily_df (DataFrame): The DataFrame containing the daily metrics.
    weekly_df (DataFrame): The DataFrame containing the weekly metrics.
    
    Returns:
    None: This function prints the LaTeX table code.
    """
    # Ensure that the input dataframes have the same structure
    assert daily_df.columns.tolist() == weekly_df.columns.tolist(), "The columns of daily_df and weekly_df must match."
    assert daily_df['Labeling Method'].tolist() == weekly_df['Labeling Method'].tolist(), "The Labeling Method rows of daily_df and weekly_df must match."
    
    # Define the LaTeX table header
    header = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        "\\begin{tabular}{|l|l|c|c|c|}\n"
        "\\hline\n"
        "\\multirow{2}{*}{\\centering Method} & \\multirow{2}{*}{Freq.} & \\multirow{2}{*}{Train Acc.} & \\multirow{2}{*}{Test Acc.} & \\multirow{2}{*}{Balanced Acc.} \\\\\n"
        " & & & & \\\\\n"
        "\\hline\n"
    )
    
    # Define the LaTeX table footer
    footer = (
        "\\end{tabular}\n"
        "\\caption{Comparison of Daily and Weekly Metrics}\n"
        "\\label{tab:daily_weekly_comparison}\n"
        "\\end{table}"
    )
    
    # Generate the table body
    body = ""
    for index, row in daily_df.iterrows():
        method = row['Labeling Method']
        daily_train_acc = row['Train Accuracy']
        daily_test_acc = row['Test Accuracy']
        daily_bal_acc = row['Balanced Accuracy']
        
        weekly_row = weekly_df[weekly_df['Labeling Method'] == method]
        weekly_train_acc = weekly_row['Train Accuracy'].values[0]
        weekly_test_acc = weekly_row['Test Accuracy'].values[0]
        weekly_bal_acc = weekly_row['Balanced Accuracy'].values[0]
        
        body += (
            f"\\multirow{{2}}{{*}}{{{method}}} & Daily & {daily_train_acc:.2f} & {daily_test_acc:.2f} & {daily_bal_acc:.2f} \\\\\n"
            f" & Weekly & {weekly_train_acc:.2f} & {weekly_test_acc:.2f} & {weekly_bal_acc:.2f} \\\\\n"
            "\\hline\n"
        )
    
    # Combine the header, body, and footer
    latex_table = header + body + footer
    
    print(latex_table)

In [10]:
def print_classification_report_to_latex(df):
    """
    This function takes a DataFrame with classification report metrics and prints a LaTeX table.
    
    Parameters:
    df (DataFrame): The DataFrame containing the classification report metrics.
    
    Returns:
    None: This function prints the LaTeX table code.
    """
    # Define the LaTeX table header
    header = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        "\\begin{tabular}{|l|c|c|c|c|c|}\n"
        "\\hline\n"
        "Method & Class & Precision & Recall & F1-Score & Support \\\\\n"
        "\\hline\n"
    )
    
    # Define the LaTeX table footer
    footer = (
        "\\end{tabular}\n"
        "\\caption{Classification Report}\n"
        "\\label{tab:classification_report}\n"
        "\\end{table}"
    )
    
    # Generate the table body
    body = ""
    previous_method = None
    method_counts = df['Method'].value_counts()
    
    for index, row in df.iterrows():
        method = row['Method']
        class_label = row['Class']
        precision = row['Precision']
        recall = row['Recall']
        f1_score = row['F1-Score']
        support = row['Support']
        
        if method != previous_method:
            method_cell = f"\\multirow{{{method_counts[method]}}}{{*}}{{{method}}}"
            previous_method = method
        else:
            method_cell = ""
        
        body += (
            f"{method_cell} & {class_label} & {precision:.2f} & {recall:.2f} & {f1_score:.2f} & {support} \\\\\n"
        )
        
        # Add \hline after the last class of each method
        if df.index[index] == df.index[-1] or df.at[index + 1, 'Method'] != method:
            body += "\\hline\n"
    
    # Combine the header, body, and footer
    latex_table = header + body + footer
    
    print(latex_table)

In [11]:
def print_financial_overview_to_latex(df):
    """
    This function takes a DataFrame with financial metrics and prints a LaTeX table.
    
    Parameters:
    df (DataFrame): The DataFrame containing the financial metrics.
    
    Returns:
    None: This function prints the LaTeX table code.
    """
    
    # Mapping of internal names to real names
    labeling_method_names = {
        'excess_over_mean': 'Excess over Mean',
        'excess_over_median': 'Excess over Median',
        'fixed_time_horizon': 'Fixed Time Horizon',
        'triple_barrier': 'Triple Barrier',
        'tail_sets': 'Tail Sets',
        'matrix_flag': 'Matrix Flag',
        'trend_scanning': 'Trend Scanning',
        'buy_and_hold': 'Buy and Hold',
        "next_period": 'Next Period Labeling'
    }

    # Define the LaTeX table header
    header = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        "\\begin{tabular}{|p{3.5cm}|c|c|c|c|}\n"
        "\\hline\n"
        "Method & TC & Overall Return & Sharpe Ratio & Max. Drawdown \\\\\n"
        "\\hline\n"
    )
    
    # Define the LaTeX table footer
    footer = (
        "\\end{tabular}\n"
        "\\caption{Financial Overview}\n"
        "\\label{tab:financial_overview}\n"
        "\\end{table}"
    )
    
    # Generate the table body
    body = ""
    for index, row in df.iterrows():
        method = labeling_method_names.get(row['labeling_method'], row['labeling_method'])
        
        # Add rows without transaction costs
        body += (
            f"\\multirow{{2}}{{*}}{{\\centering {method}}} & No & {row['overall_return']:.2f} & {row['sharpe_ratio']:.2f} & {row['maximum_drawdown']:.2f} \\\\\n"
        )
        
        # Add rows with transaction costs
        body += (
            f" & Yes & {row['overall_return_TC']:.2f} & {row['sharpe_ratio_TC']:.2f} & {row['maximum_drawdown_TC']:.2f} \\\\\n"
            "\\hline\n"
        )
    
    # Combine the header, body, and footer
    latex_table = header + body + footer
    
    print(latex_table)

In [12]:
def print_holding_periods_to_latex(df):
    """
    This function takes a DataFrame with financial metrics and prints a LaTeX table.
    
    Parameters:
    df (DataFrame): The DataFrame containing the financial metrics.
    
    Returns:
    None: This function prints the LaTeX table code.
    """
    
    # Mapping of internal names to real names
    labeling_method_names = {
        'excess_over_mean': 'Excess over Mean',
        'excess_over_median': 'Excess over Median',
        'fixed_time_horizon': 'Fixed Time Horizon',
        'triple_barrier': 'Triple Barrier',
        'tail_sets': 'Tail Sets',
        'matrix_flag': 'Matrix Flag',
        'trend_scanning': 'Trend Scanning',
        'buy_and_hold': 'Buy and Hold',
        "next_period": 'Next Period Labeling'
    }

    # Check if required columns exist in the DataFrame
    required_columns = ['labeling_method', 'currency', 'return_3_days', 'return_10_days', 'return_30_days']
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' not found in DataFrame")

    # Filter the DataFrame to include only rows where currency is 'overall'
    df = df[df['currency'] == 'overall']

    # Define the LaTeX table header
    header = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        "\\begin{tabular}{|>{\\centering\\arraybackslash}p{3.5cm}|>{\\centering\\arraybackslash}p{2.5cm}|>{\\centering\\arraybackslash}p{2.5cm}|>{\\centering\\arraybackslash}p{2.5cm}|}\n"
        "\\hline\n"
        "Method & 3-Day Return & 10-Day Return & 30-Day Return \\\\\n"
        "\\hline\n"
    )
    
    # Define the LaTeX table footer
    footer = (
        "\\end{tabular}\n"
        "\\caption{Financial Overview}\n"
        "\\label{tab:financial_overview}\n"
        "\\end{table}"
    )
    
    # Generate the table body
    body = ""
    for index, row in df.iterrows():
        method = labeling_method_names.get(row['labeling_method'], row['labeling_method'])
        
        # Add rows
        body += (
            f"{method} & {row['return_3_days']:.2f} & {row['return_10_days']:.2f} & {row['return_30_days']:.2f} \\\\\n"
            "\\hline\n"
        )
    
    # Combine the header, body, and footer
    latex_table = header + body + footer
    
    print(latex_table)

In [13]:
def print_trade_metrics_to_latex(df):
    """
    This function takes a DataFrame with trade metrics and prints three LaTeX tables.
    
    Parameters:
    df (DataFrame): The DataFrame containing the trade metrics.
    
    Returns:
    None: This function prints the LaTeX table codes.
    """
    
    # Mapping of internal names to real names
    labeling_method_names = {
        'excess_over_mean': 'Excess over Mean',
        'excess_over_median': 'Excess over Median',
        'fixed_time_horizon': 'Fixed Time Horizon',
        'triple_barrier': 'Triple Barrier',
        'tail_sets': 'Tail Sets',
        'matrix_flag': 'Matrix Flag',
        'trend_scanning': 'Trend Scanning',
        'buy_and_hold': 'Buy and Hold',
        "next_period": 'Next Period Labeling'
    }

    # Ensure all required columns are present, fill missing columns with default values
    required_columns_defaults = {
        'Labeling Method': '',
        'Profitable Trades': 0,
        'Profitable Trades after TC': 0,
        'Percentage of Profitable Trades Total': 0.0,
        'Percentage of Profitable Trades Total after TC': 0.0,
        'Trades with >5% Profit': 0,
        'Trades with >5% Loss': 0,
        'Trades with >10% Profit': 0,
        'Trades with >10% Loss': 0,
        'Average Return per Trade': 0.0,
        'Average Return per Period': 0.0,
        'Median Return per Trade': 0.0,
        'Median Return per Period': 0.0
    }

    for col, default in required_columns_defaults.items():
        if col not in df.columns:
            df[col] = default

    # Define the LaTeX table headers and footers
    headers = {
        "table1": (
            "\\begin{table}[ht]\n"
            "\\centering\n"
            "\\begin{tabular}{|p{3.5cm}|c|c|c|c|}\n"
            "\\hline\n"
            "Labeling Method & Profitable Trades & Profitable Trades after TC & "
            "Percentage of Profitable Trades Total & Percentage of Profitable Trades Total after TC \\\\\n"
            "\\hline\n"
        ),
        "table2": (
            "\\begin{table}[ht]\n"
            "\\centering\n"
            "\\begin{tabular}{|p{3.5cm}|c|c|c|c|}\n"
            "\\hline\n"
            "Labeling Method & Trades with >5\% Profit & Trades with >5\% Loss & "
            "Trades with >10\% Profit & Trades with >10\% Loss \\\\\n"
            "\\hline\n"
        ),
        "table3": (
            "\\begin{table}[ht]\n"
            "\\centering\n"
            "\\begin{tabular}{|p{3.5cm}|c|c|c|c|}\n"
            "\\hline\n"
            "Labeling Method & Average Return per Trade & Average Return per Period & "
            "Median Return per Trade & Median Return per Period \\\\\n"
            "\\hline\n"
        ),
    }

    footers = {
        "table1": (
            "\\end{tabular}\n"
            "\\caption{Profitable Trades Overview}\n"
            "\\label{tab:profitable_trades_overview}\n"
            "\\end{table}"
        ),
        "table2": (
            "\\end{tabular}\n"
            "\\caption{Profit and Loss Trades Overview}\n"
            "\\label{tab:profit_loss_trades_overview}\n"
            "\\end{table}"
        ),
        "table3": (
            "\\end{tabular}\n"
            "\\caption{Return Metrics Overview}\n"
            "\\label{tab:return_metrics_overview}\n"
            "\\end{table}"
        ),
    }

    # Define the table body generators
    def generate_table_body(df, columns):
        body = ""
        for index, row in df.iterrows():
            method = labeling_method_names.get(row['Labeling Method'], row['Labeling Method'])
            row_data = [method] + [f"{row[col]:.2f}" if isinstance(row[col], (int, float)) else str(row[col]) for col in columns[1:]]
            body += " & ".join(row_data) + " \\\\\n"
            body += "\\hline\n"
        return body

    # Generate and print the LaTeX tables
    for table, columns in zip(
        ["table1", "table2", "table3"], 
        [
            ['Labeling Method', 'Profitable Trades', 'Profitable Trades after TC', 'Percentage of Profitable Trades Total', 'Percentage of Profitable Trades Total after TC'],
            ['Labeling Method', 'Trades with >5% Profit', 'Trades with >5% Loss', 'Trades with >10% Profit', 'Trades with >10% Loss'],
            ['Labeling Method', 'Average Return per Trade', 'Average Return per Period', 'Median Return per Trade', 'Median Return per Period']
        ]
    ):
        header = headers[table]
        footer = footers[table]
        body = generate_table_body(df, columns)
        latex_table = header + body + footer
        print(latex_table)
        print("---------------------")

In [14]:
def print_trade_metrics_to_latex(df):
    """
    This function takes a DataFrame with trade metrics and prints three LaTeX tables.
    
    Parameters:
    df (DataFrame): The DataFrame containing the trade metrics.
    
    Returns:
    None: This function prints the LaTeX table codes.
    """
    
    # Mapping of internal names to real names
    labeling_method_names = {
        'excess_over_mean': 'Excess over Mean',
        'excess_over_median': 'Excess over Median',
        'fixed_time_horizon': 'Fixed Time Horizon',
        'triple_barrier': 'Triple Barrier',
        'tail_sets': 'Tail Sets',
        'matrix_flag': 'Matrix Flag',
        'trend_scanning': 'Trend Scanning',
        'buy_and_hold': 'Buy and Hold',
        "next_period": 'Next Period Labeling'
    }

    # Ensure all required columns are present, fill missing columns with default values
    required_columns_defaults = {
        'Labeling Method': '',
        'Profitable Trades': 0,
        'Profitable Trades after TC': 0,
        'Percentage of Profitable Trades Total': 0.0,
        'Percentage of Profitable Trades Total after TC': 0.0,
        'Trades with >5% Profit': 0,
        'Trades with >5% Loss': 0,
        'Trades with >10% Profit': 0,
        'Trades with >10% Loss': 0,
        'Average Return per Trade': 0.0,
        'Average Return per Period': 0.0,
        'Median Return per Trade': 0.0,
        'Median Return per Period': 0.0,
        'Trades with >1% Profit': 0,
        'Trades with >1% Loss': 0,
        'Trades with >0.6% Profit': 0,
        'Trades with >0.6% Loss': 0
    }

    for col, default in required_columns_defaults.items():
        if col not in df.columns:
            df[col] = default

    # Define the LaTeX table headers and footers
    headers = {
        "table1": (
            "\\begin{table}[ht]\n"
            "\\centering\n"
            "\\begin{tabular}{|p{3.5cm}|c|c|c|c|}\n"
            "\\hline\n"
            "Labeling Method & Profitable Trades & Profitable Trades after TC & "
            "Profitable Total & Profitable Total after TC \\\\\n"
            "\\hline\n"
        ),
        "table2": (
            "\\begin{table}[ht]\n"
            "\\centering\n"
            "\\begin{tabular}{|p{3.5cm}|c|c|c|c|c|c|c|c|}\n"
            "\\hline\n"
            "Labeling Method & >5\% & <5\% & >10\% & <10\% & >1\% & <1\% & >0.6\% & <0.6\% \\\\\n"
            "\\hline\n"
        ),
        "table3": (
            "\\begin{table}[ht]\n"
            "\\centering\n"
            "\\begin{tabular}{|p{3.5cm}|c|c|c|c|}\n"
            "\\hline\n"
            "Labeling Method & Avg Return per Trade & Avg Return per Period & "
            "Median Return per Trade & Median Return per Period \\\\\n"
            "\\hline\n"
        ),
    }

    footers = {
        "table1": (
            "\\end{tabular}\n"
            "\\caption{Profitable Trades Overview}\n"
            "\\label{tab:profitable_trades_overview}\n"
            "\\end{table}"
        ),
        "table2": (
            "\\end{tabular}\n"
            "\\caption{Profit and Loss Trades Overview}\n"
            "\\label{tab:profit_loss_trades_overview}\n"
            "\\end{table}"
        ),
        "table3": (
            "\\end{tabular}\n"
            "\\caption{Return Metrics Overview}\n"
            "\\label{tab:return_metrics_overview}\n"
            "\\end{table}"
        ),
    }

    # Define the table body generators
    def generate_table_body(df, columns):
        body = ""
        for index, row in df.iterrows():
            method = labeling_method_names.get(row['Labeling Method'], row['Labeling Method'])
            row_data = [method] + [f"{row[col]:.2f}" if isinstance(row[col], (int, float)) else str(row[col]) for col in columns[1:]]
            body += " & ".join(row_data) + " \\\\\n"
            body += "\\hline\n"
        return body

    # Generate and print the LaTeX tables
    for table, columns in zip(
        ["table1", "table2", "table3"], 
        [
            ['Labeling Method', 'Profitable Trades', 'Profitable Trades after TC', 'Percentage of Profitable Trades Total', 'Percentage of Profitable Trades Total after TC'],
            ['Labeling Method', 'Trades with >5% Profit', 'Trades with >5% Loss', 'Trades with >10% Profit', 'Trades with >10% Loss',
             'Trades with >1% Profit', 'Trades with >1% Loss', 'Trades with >0.6% Profit', 'Trades with >0.6% Loss'],
            ['Labeling Method', 'Average Return per Trade', 'Average Return per Period', 'Median Return per Trade', 'Median Return per Period']
        ]
    ):
        header = headers[table]
        footer = footers[table]
        body = generate_table_body(df, columns)
        latex_table = header + body + footer
        print(latex_table)
        print("---------------------")

In [15]:
import pandas as pd

def print_trade_metrics_to_latex(daily_df, weekly_df):
    """
    This function takes two DataFrames (daily and weekly) with trade metrics and prints three LaTeX tables comparing their metrics.
    
    Parameters:
    daily_df (DataFrame): The DataFrame containing the daily trade metrics.
    weekly_df (DataFrame): The DataFrame containing the weekly trade metrics.
    
    Returns:
    None: This function prints the LaTeX table codes.
    """
    
    # Ensure that the input dataframes have the same structure
    assert daily_df.columns.tolist() == weekly_df.columns.tolist(), "The columns of daily_df and weekly_df must match."
    assert daily_df['Labeling Method'].tolist() == weekly_df['Labeling Method'].tolist(), "The Labeling Method rows of daily_df and weekly_df must match."
    
    # Mapping of internal names to real names
    labeling_method_names = {
        'excess_over_mean': 'Excess over Mean',
        'excess_over_median': 'Excess over Median',
        'fixed_time_horizon': 'Fixed Time Horizon',
        'triple_barrier': 'Triple Barrier',
        'tail_sets': 'Tail Sets',
        'matrix_flag': 'Matrix Flag',
        'trend_scanning': 'Trend Scanning',
        'buy_and_hold': 'Buy and Hold',
        "next_period": 'Next Period Labeling'
    }

    # Ensure all required columns are present, fill missing columns with default values
    required_columns_defaults = {
        'Labeling Method': '',
        'Profitable Trades': 0,
        'Profitable Trades after TC': 0,
        'Percentage of Profitable Trades Total': 0.0,
        'Percentage of Profitable Trades Total after TC': 0.0,
        'Trades with >5% Profit': 0,
        'Trades with >5% Loss': 0,
        'Trades with >10% Profit': 0,
        'Trades with >10% Loss': 0,
        'Average Return per Trade': 0.0,
        'Average Return per Period': 0.0,
        'Median Return per Trade': 0.0,
        'Median Return per Period': 0.0,
        'Trades with >1% Profit': 0,
        'Trades with >1% Loss': 0,
        'Trades with >0.6% Profit': 0,
        'Trades with >0.6% Loss': 0
    }

    for col, default in required_columns_defaults.items():
        if col not in daily_df.columns:
            daily_df[col] = default
        if col not in weekly_df.columns:
            weekly_df[col] = default

    # Define the LaTeX table headers and footers
    headers = {
        "table1": (
            "\\begin{table}[ht]\n"
            "\\centering\n"
            "\\begin{tabular}{|p{3.5cm}|c|c|c|c|c|}\n"
            "\\hline\n"
            "Labeling Method & Freq. & Profitable Trades & Profitable Trades after TC & "
            "Profitable Total & Profitable Total after TC \\\\\n"
            "\\hline\n"
        ),
        "table2": (
            "\\begin{table}[ht]\n"
            "\\centering\n"
            "\\begin{tabular}{|p{3.5cm}|c|c|c|c|c|c|c|c|c|}\n"
            "\\hline\n"
            "Labeling Method & Freq. & >5\% & <5\% & >10\% & <10\% & >1\% & <1\% & >0.6\% & <0.6\% \\\\\n"
            "\\hline\n"
        ),
        "table3": (
            "\\begin{table}[ht]\n"
            "\\centering\n"
            "\\begin{tabular}{|p{3.5cm}|c|c|c|c|c|}\n"
            "\\hline\n"
            "Labeling Method & Freq. & Avg Return per Trade & Avg Return per Period & "
            "Median Return per Trade & Median Return per Period \\\\\n"
            "\\hline\n"
        ),
    }

    footers = {
        "table1": (
            "\\end{tabular}\n"
            "\\caption{Profitable Trades Overview}\n"
            "\\label{tab:profitable_trades_overview}\n"
            "\\end{table}"
        ),
        "table2": (
            "\\end{tabular}\n"
            "\\caption{Profit and Loss Trades Overview}\n"
            "\\label{tab:profit_loss_trades_overview}\n"
            "\\end{table}"
        ),
        "table3": (
            "\\end{tabular}\n"
            "\\caption{Return Metrics Overview}\n"
            "\\label{tab:return_metrics_overview}\n"
            "\\end{table}"
        ),
    }

    # Define the table body generators
    def generate_table_body(daily_df, weekly_df, columns):
        body = ""
        for index, row in daily_df.iterrows():
            method = labeling_method_names.get(row['Labeling Method'], row['Labeling Method'])
            
            # Extract daily metrics
            daily_metrics = [f"{row[col]:.2f}" if isinstance(row[col], (int, float)) else str(row[col]) for col in columns[2:]]
            
            # Find corresponding weekly metrics
            weekly_row = weekly_df[weekly_df['Labeling Method'] == row['Labeling Method']].iloc[0]
            weekly_metrics = [f"{weekly_row[col]:.2f}" if isinstance(weekly_row[col], (int, float)) else str(weekly_row[col]) for col in columns[2:]]
            
            # Combine daily and weekly metrics for the same labeling method
            body += f"\\multirow{{2}}{{*}}{{{method}}} & Daily & " + " & ".join(daily_metrics) + " \\\\\n"
            body += f" & Weekly & " + " & ".join(weekly_metrics) + " \\\\\n"
            body += "\\hline\n"
        
        return body

    # Generate and print the LaTeX tables
    for table, columns in zip(
        ["table1", "table2", "table3"], 
        [
            ['Labeling Method', 'Freq.', 'Profitable Trades', 'Profitable Trades after TC', 'Percentage of Profitable Trades Total', 'Percentage of Profitable Trades Total after TC'],
            ['Labeling Method', 'Freq.', 'Trades with >5% Profit', 'Trades with >5% Loss', 'Trades with >10% Profit', 'Trades with >10% Loss',
             'Trades with >1% Profit', 'Trades with >1% Loss', 'Trades with >0.6% Profit', 'Trades with >0.6% Loss'],
            ['Labeling Method', 'Freq.', 'Average Return per Trade', 'Average Return per Period', 'Median Return per Trade', 'Median Return per Period']
        ]
    ):
        header = headers[table]
        footer = footers[table]
        body = generate_table_body(daily_df, weekly_df, columns)
        latex_table = header + body + footer
        print(latex_table)
        print("---------------------")

In [16]:
def print_trade_metrics_to_latex(daily_df, weekly_df):
    """
    This function takes two DataFrames (daily and weekly) with trade metrics and prints three LaTeX tables comparing their metrics.
    
    Parameters:
    daily_df (DataFrame): The DataFrame containing the daily trade metrics.
    weekly_df (DataFrame): The DataFrame containing the weekly trade metrics.
    
    Returns:
    None: This function prints the LaTeX table codes.
    """
    
    # Ensure that the input dataframes have the same structure
    assert daily_df.columns.tolist() == weekly_df.columns.tolist(), "The columns of daily_df and weekly_df must match."
    assert daily_df['Labeling Method'].tolist() == weekly_df['Labeling Method'].tolist(), "The Labeling Method rows of daily_df and weekly_df must match."
    
    # Mapping of internal names to real names
    labeling_method_names = {
        'excess_over_mean': 'Excess over Mean',
        'excess_over_median': 'Excess over Median',
        'fixed_time_horizon': 'Fixed Time Horizon',
        'triple_barrier': 'Triple Barrier',
        'tail_sets': 'Tail Sets',
        'matrix_flag': 'Matrix Flag',
        'trend_scanning': 'Trend Scanning',
        'buy_and_hold': 'Buy and Hold',
        "next_period": 'Next Period Labeling'
    }

    # Ensure all required columns are present, fill missing columns with default values
    required_columns_defaults = {
        'Labeling Method': '',
        'Profitable Trades': 0,
        'Profitable Trades TC': 0,
        'Profitable Periods': 0,
        '>0.6%': 0,
        '>1%': 0,
        '>5%': 0,
        '>10%': 0,
        'Average Return per Trade': 0.0,
        'Average Return per Period': 0.0,
        'Median Return per Trade': 0.0,
        'Median Return per Period': 0.0
    }

    for col, default in required_columns_defaults.items():
        if col not in daily_df.columns:
            daily_df[col] = default
        if col not in weekly_df.columns:
            weekly_df[col] = default

    # Define the LaTeX table headers and footers
    headers = {
        "table1": (
            "\\begin{table}[ht]\n"
            "\\centering\n"
            "\\begin{tabular}{|p{3.5cm}|c|c|c|c|}\n"
            "\\hline\n"
            "Labeling Method & Freq. & Profitable Trades & Profitable Trades TC & Profitable Periods \\\\\n"
            "\\hline\n"
        ),
        "table2": (
            "\\begin{table}[ht]\n"
            "\\centering\n"
            "\\begin{tabular}{|p{3.5cm}|c|c|c|c|c|}\n"
            "\\hline\n"
            "Labeling Method & Freq. & $>0.6\%$ & $>1\%$ & $>5\%$ & $>10\%$ \\\\\n"
            "\\hline\n"
        ),
        "table3": (
            "\\begin{table}[ht]\n"
            "\\centering\n"
            "\\begin{tabular}{|p{3.5cm}|c|c|c|c|c|}\n"
            "\\hline\n"
            "Labeling Method & Freq. & $\\mu$ Trade & $\\mu$ Period & $\\tilde{x}$ Trade & $\\tilde{x}$ Period \\\\\n"
            "\\hline\n"
        )
    }

    footers = {
        "table1": (
            "\\end{tabular}\n"
            "\\caption{Profitable Trades Overview}\n"
            "\\label{tab:profitable_trades_overview}\n"
            "\\end{table}"
        ),
        "table2": (
            "\\end{tabular}\n"
            "\\caption{Profit and Loss Trades Overview}\n"
            "\\label{tab:profit_loss_trades_overview}\n"
            "\\end{table}"
        ),
        "table3": (
            "\\end{tabular}\n"
            "\\caption{Return Metrics Overview}\n"
            "\\label{tab:return_metrics_overview}\n"
            "\\end{table}"
        )
    }

    # Define the table body generators
    def generate_table_body(daily_df, weekly_df, columns):
        body = ""
        for index, row in daily_df.iterrows():
            method = labeling_method_names.get(row['Labeling Method'], row['Labeling Method'])
            
            # Extract daily metrics
            daily_metrics = [f"{row[col]:.2f}" if isinstance(row[col], (int, float)) and row[col] != 0 else "-" for col in columns[2:]]
            
            # Find corresponding weekly metrics
            weekly_row = weekly_df[weekly_df['Labeling Method'] == row['Labeling Method']].iloc[0]
            weekly_metrics = [f"{weekly_row[col]:.2f}" if isinstance(weekly_row[col], (int, float)) and weekly_row[col] != 0 else "-" for col in columns[2:]]
            
            # Combine daily and weekly metrics for the same labeling method
            body += f"\\multirow{{2}}{{*}}{{{method}}} & Daily & " + " & ".join(daily_metrics) + " \\\\\n"
            body += f" & Weekly & " + " & ".join(weekly_metrics) + " \\\\\n"
            body += "\\hline\n"
        
        return body

    # Generate and print the LaTeX tables
    for table, columns in zip(
        ["table1", "table2", "table3"], 
        [
            ['Labeling Method', 'Freq.', 'Profitable Trades', 'Profitable Trades TC', 'Profitable Periods'],
            ['Labeling Method', 'Freq.', '>0.6%', '>1%', '>5%', '>10%'],
            ['Labeling Method', 'Freq.', 'Average Return per Trade', 'Average Return per Period', 'Median Return per Trade', 'Median Return per Period']
        ]
    ):
        header = headers[table]
        footer = footers[table]
        body = generate_table_body(daily_df, weekly_df, columns)
        latex_table = header + body + footer
        print(latex_table)
        print("---------------------")

In [17]:
# Mapping of internal names to real names
labeling_method_names = {
    'excess_over_mean': 'Excess over Mean',
    'excess_over_median': 'Excess over Median',
    'fixed_time_horizon': 'Fixed Time Horizon',
    'triple_barrier': 'Triple Barrier',
    'tail_sets': 'Tail Sets',
    'matrix_flag': 'Matrix Flag',
    'trend_scanning': 'Trend Scanning',
    'buy_and_hold': 'Buy and Hold',
    'next_period': 'Next Period Labeling'
}

def rename_columns(df):
    """
    Rename columns for better clarity.
    """
    return df.rename(columns={
        'Labeling Method': 'Method',
        'Profitable Trades': 'Prof. Trades',
        'Profitable Trades after TC': 'Prof. Trades (TC)',
        'Percentage of Profitable Trades Total': 'Prof. Periods',
        'Percentage of Profitable Trades Total after TC': 'Prof. Periods (TC)',
        'Trades with >10% Profit': 'Trades > 10%',
        'Trades with >10% Loss': 'Trades < -10%',
        'Trades with >5% Profit': 'Trades > 5%',
        'Trades with >5% Loss': 'Trades < -5%',
        'Trades with >1% Profit': 'Trades > 1%',
        'Trades with >1% Loss': 'Trades < -1%',
        'Trades with >0.6% Profit': 'Trades > 0.6%',
        'Trades with >0.6% Loss': 'Trades < -0.6%',
        'Average Return per Trade': 'Avg. Trade',
        'Average Return per Period': 'Avg. Period',
        'Median Return per Trade': 'Med. Trade',
        'Median Return per Period': 'Med. Period'
    })

def map_labeling_methods(df):
    """
    Map internal labeling method names to real names.
    """
    df['Method'] = df['Method'].map(labeling_method_names)
    return df

def format_to_latex(df, title, label):
    """
    Format DataFrame to LaTeX table with merged cells.
    """
    latex_code = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        "\\begin{tabular}{|l|c|c|c|c|}\n"
        "\\hline\n"
        "Method & Frequency & Prof. Trades & Prof. Trades (TC) & Prof. Periods \\\\\n"
        "\\hline\n"
    )
    
    current_method = None
    for _, row in df.iterrows():
        if row['Method'] != current_method:
            if current_method is not None:
                latex_code += "\\hline\n"
            current_method = row['Method']
            latex_code += f"\\multirow{{2}}{{*}}{{{row['Method']}}} & {row['Frequency']} & {row['Prof. Trades']:.2f} & {row['Prof. Trades (TC)']:.2f} & {row['Prof. Periods']:.2f} \\\\\n"
        else:
            latex_code += f" & {row['Frequency']} & {row['Prof. Trades']:.2f} & {row['Prof. Trades (TC)']:.2f} & {row['Prof. Periods']:.2f} \\\\\n"
    
    latex_code += (
        "\\hline\n"
        "\\end{tabular}\n"
        f"\\caption{{{title}}}\n"
        f"\\label{{{label}}}\n"
        "\\end{table}"
    )
    
    return latex_code

def print_trade_metrics_to_latex(daily_df, weekly_df):
    """
    This function takes two DataFrames (daily and weekly) with trade metrics and prints LaTeX tables comparing their metrics.
    
    Parameters:
    daily_df (DataFrame): The DataFrame containing the daily trade metrics.
    weekly_df (DataFrame): The DataFrame containing the weekly trade metrics.
    
    Returns:
    None: This function prints the LaTeX table codes.
    """
    # Rename columns
    daily_df = rename_columns(daily_df)
    weekly_df = rename_columns(weekly_df)
    
    # Map labeling methods
    daily_df = map_labeling_methods(daily_df)
    weekly_df = map_labeling_methods(weekly_df)
    
    # Combine daily and weekly DataFrames
    daily_df['Frequency'] = 'Daily'
    weekly_df['Frequency'] = 'Weekly'
    
    combined_df = pd.concat([daily_df, weekly_df], ignore_index=True)
    
    # Sort by method to ensure correct order
    combined_df.sort_values(by='Method', inplace=True)
    
    # Generate LaTeX table for Profitable Trades
    latex_table_1 = format_to_latex(combined_df, "Profitable Trades Overview", "tab:profitable_trades_overview")
    print(latex_table_1)
    print("---------------------")

In [18]:
# Mapping of internal names to real names
labeling_method_names = {
    'excess_over_mean': 'Excess over Mean',
    'excess_over_median': 'Excess over Median',
    'fixed_time_horizon': 'Fixed Time Horizon',
    'triple_barrier': 'Triple Barrier',
    'tail_sets': 'Tail Sets',
    'matrix_flag': 'Matrix Flag',
    'trend_scanning': 'Trend Scanning',
    'buy_and_hold': 'Buy and Hold',
    'next_period': 'Next Period Labeling'
}

def rename_columns(df):
    """
    Rename columns for better clarity.
    """
    return df.rename(columns={
        'Labeling Method': 'Method',
        'Profitable Trades': 'Prof. Trades',
        'Profitable Trades after TC': 'Prof. Trades (TC)',
        'Percentage of Profitable Trades Total': 'Prof. Periods',
        'Percentage of Profitable Trades Total after TC': 'Prof. Periods (TC)',
        'Trades with >10% Profit': 'Trades > 10%',
        'Trades with >10% Loss': 'Trades < -10%',
        'Trades with >5% Profit': 'Trades > 5%',
        'Trades with >5% Loss': 'Trades < -5%',
        'Trades with >1% Profit': 'Trades > 1%',
        'Trades with >1% Loss': 'Trades < -1%',
        'Trades with >0.6% Profit': 'Trades > 0.6%',
        'Trades with >0.6% Loss': 'Trades < -0.6%',
        'Average Return per Trade': 'Avg. Trade',
        'Average Return per Period': 'Avg. Period',
        'Median Return per Trade': 'Med. Trade',
        'Median Return per Period': 'Med. Period'
    })

def map_labeling_methods(df):
    """
    Map internal labeling method names to real names.
    """
    df['Method'] = df['Method'].map(labeling_method_names)
    return df

def format_to_latex(df, title, label, columns, headers):
    """
    Format DataFrame to LaTeX table with merged cells.
    """
    latex_code = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        "\\begin{tabular}{|p{3.5cm}|c|c|c|c|c|}\n"
        "\\hline\n"
        + headers +
        "\\hline\n"
    )
    
    current_method = None
    for _, row in df.iterrows():
        if row['Method'] != current_method:
            if current_method is not None:
                latex_code += "\\hline\n"
            current_method = row['Method']
            latex_code += f"\\multirow{{2}}{{*}}{{{row['Method']}}} & {row['Frequency']} & " + " & ".join([f"{row[col]:.2f}" for col in columns[2:]]) + " \\\\\n"
        else:
            latex_code += f" & {row['Frequency']} & " + " & ".join([f"{row[col]:.2f}" for col in columns[2:]]) + " \\\\\n"
    
    latex_code += (
        "\\hline\n"
        "\\end{tabular}\n"
        f"\\caption{{{title}}}\n"
        f"\\label{{{label}}}\n"
        "\\end{table}"
    )
    
    return latex_code

def print_trade_metrics_to_latex(daily_df, weekly_df):
    """
    This function takes two DataFrames (daily and weekly) with trade metrics and prints LaTeX tables comparing their metrics.
    
    Parameters:
    daily_df (DataFrame): The DataFrame containing the daily trade metrics.
    weekly_df (DataFrame): The DataFrame containing the weekly trade metrics.
    
    Returns:
    None: This function prints the LaTeX table codes.
    """
    # Rename columns
    daily_df = rename_columns(daily_df)
    weekly_df = rename_columns(weekly_df)
    
    # Map labeling methods
    daily_df = map_labeling_methods(daily_df)
    weekly_df = map_labeling_methods(weekly_df)
    
    # Combine daily and weekly DataFrames
    daily_df['Frequency'] = 'Daily'
    weekly_df['Frequency'] = 'Weekly'
    
    combined_df = pd.concat([daily_df, weekly_df], ignore_index=True)
    
    # Sort by method to ensure correct order
    combined_df.sort_values(by='Method', inplace=True)
    
    # Generate LaTeX table for Profitable Trades
    latex_table_1 = format_to_latex(
        combined_df, 
        "Profitable Trades Overview", 
        "tab:profitable_trades_overview",
        ['Method', 'Frequency', 'Prof. Trades', 'Prof. Trades (TC)', 'Prof. Periods'],
        "Method & Freq. & Prof. Trades & Prof. Trades (TC) & Prof. Periods \\\\\n"
    )
    print(latex_table_1)
    print("---------------------")
    
    # Generate LaTeX table for Profit and Loss Trades
    latex_table_2 = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        "\\begin{tabular}{|p{3.5cm}|c|c|c|c|c|}\n"
        "\\hline\n"
        "Method & Freq. & $>0.6\\%$ & $>1\\%$ & $>5\\%$ & $>10\\%$ \\\\\n"
        "\\hline\n"
    )
    
    current_method = None
    for _, row in combined_df.iterrows():
        if row['Method'] != current_method:
            if current_method is not None:
                latex_table_2 += "\\hline\n"
            current_method = row['Method']
            latex_table_2 += f"\\multirow{{2}}{{*}}{{{row['Method']}}} & {row['Frequency']} & " + " & ".join([f"{row[col]:.2f}" for col in ['Trades > 0.6%', 'Trades > 1%', 'Trades > 5%', 'Trades > 10%']]) + " \\\\\n"
        else:
            latex_table_2 += f" & {row['Frequency']} & " + " & ".join([f"{row[col]:.2f}" for col in ['Trades > 0.6%', 'Trades > 1%', 'Trades > 5%', 'Trades > 10%']]) + " \\\\\n"
    
    latex_table_2 += (
        "\\hline\n"
        "\\end{tabular}\n"
        "\\caption{Profit and Loss Trades Overview}\n"
        "\\label{tab:profit_loss_trades_overview}\n"
        "\\end{table}"
    )
    print(latex_table_2)
    print("---------------------")
    
    # Generate LaTeX table for Return Metrics
    latex_table_3 = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        "\\begin{tabular}{|p{3.5cm}|c|c|c|c|c|}\n"
        "\\hline\n"
        "Method & Freq. & $\\mu$ Trade & $\\mu$ Period & $\\tilde{x}$ Trade & $\\tilde{x}$ Period \\\\\n"
        "\\hline\n"
    )
    
    current_method = None
    for _, row in combined_df.iterrows():
        if row['Method'] != current_method:
            if current_method is not None:
                latex_table_3 += "\\hline\n"
            current_method = row['Method']
            latex_table_3 += f"\\multirow{{2}}{{*}}{{{row['Method']}}} & {row['Frequency']} & " + " & ".join([f"{row[col]:.2f}" for col in ['Avg. Trade', 'Avg. Period', 'Med. Trade', 'Med. Period']]) + " \\\\\n"
        else:
            latex_table_3 += f" & {row['Frequency']} & " + " & ".join([f"{row[col]:.2f}" for col in ['Avg. Trade', 'Avg. Period', 'Med. Trade', 'Med. Period']]) + " \\\\\n"
    
    latex_table_3 += (
        "\\hline\n"
        "\\end{tabular}\n"
        "\\caption{Return Metrics Overview}\n"
        "\\label{tab:return_metrics_overview}\n"
        "\\end{table}"
    )
    print(latex_table_3)
    print("---------------------")

In [19]:
# Mapping of internal names to real names
labeling_method_names = {
    'excess_over_mean': 'Excess over Mean',
    'excess_over_median': 'Excess over Median',
    'fixed_time_horizon': 'Fixed Time Horizon',
    'triple_barrier': 'Triple Barrier',
    'tail_sets': 'Tail Sets',
    'matrix_flag': 'Matrix Flag',
    'trend_scanning': 'Trend Scanning',
    'buy_and_hold': 'Buy and Hold',
    'next_period': 'Next Period Labeling'
}

def rename_columns(df):
    """
    Rename columns for better clarity.
    """
    return df.rename(columns={
        'Labeling Method': 'Method',
        'Profitable Trades': 'Prof. Trades',
        'Profitable Trades after TC': 'Prof. Trades (TC)',
        'Percentage of Profitable Trades Total': 'Prof. Periods',
        'Percentage of Profitable Trades Total after TC': 'Prof. Periods (TC)',
        'Trades with >10% Profit': 'Trades > 10%',
        'Trades with >10% Loss': 'Trades < -10%',
        'Trades with >5% Profit': 'Trades > 5%',
        'Trades with >5% Loss': 'Trades < -5%',
        'Trades with >1% Profit': 'Trades > 1%',
        'Trades with >1% Loss': 'Trades < -1%',
        'Trades with >0.6% Profit': 'Trades > 0.6%',
        'Trades with >0.6% Loss': 'Trades < -0.6%',
        'Average Return per Trade': 'Avg. Trade',
        'Average Return per Period': 'Avg. Period',
        'Median Return per Trade': 'Med. Trade',
        'Median Return per Period': 'Med. Period'
    })

def map_labeling_methods(df):
    """
    Map internal labeling method names to real names.
    """
    df['Method'] = df['Method'].map(labeling_method_names)
    return df

def format_to_latex(df, title, label, columns, headers):
    """
    Format DataFrame to LaTeX table with merged cells.
    """
    latex_code = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        f"\\begin{{tabular}}{{|{'|'.join(['p{3.5cm}'] + ['c']*len(columns[2:]))}|}}\n"
        "\\hline\n"
        + headers +
        "\\hline\n"
    )
    
    current_method = None
    for _, row in df.iterrows():
        if row['Method'] != current_method:
            if current_method is not None:
                latex_code += "\\hline\n"
            current_method = row['Method']
            latex_code += f"\\multirow{{2}}{{*}}{{{row['Method']}}} & {row['Frequency']} & " + " & ".join([f"{row[col]:.2f}" if abs(row[col]) >= 1e-2 else "0.00" for col in columns[2:]]) + " \\\\\n"
        else:
            latex_code += f" & {row['Frequency']} & " + " & ".join([f"{row[col]:.2f}" if abs(row[col]) >= 1e-2 else "0.00" for col in columns[2:]]) + " \\\\\n"
    
    latex_code += (
        "\\hline\n"
        "\\end{tabular}\n"
        f"\\caption{{{title}}}\n"
        f"\\label{{{label}}}\n"
        "\\end{table}"
    )
    
    return latex_code

def print_trade_metrics_to_latex(daily_df, weekly_df):
    """
    This function takes two DataFrames (daily and weekly) with trade metrics and prints LaTeX tables comparing their metrics.
    
    Parameters:
    daily_df (DataFrame): The DataFrame containing the daily trade metrics.
    weekly_df (DataFrame): The DataFrame containing the weekly trade metrics.
    
    Returns:
    None: This function prints the LaTeX table codes.
    """
    # Rename columns
    daily_df = rename_columns(daily_df)
    weekly_df = rename_columns(weekly_df)
    
    # Map labeling methods
    daily_df = map_labeling_methods(daily_df)
    weekly_df = map_labeling_methods(weekly_df)
    
    # Combine daily and weekly DataFrames
    daily_df['Frequency'] = 'Daily'
    weekly_df['Frequency'] = 'Weekly'
    
    combined_df = pd.concat([daily_df, weekly_df], ignore_index=True)
    
    # Sort by method to ensure correct order
    combined_df.sort_values(by='Method', inplace=True)
    
    # Generate LaTeX table for Profitable Trades
    latex_table_1 = format_to_latex(
        combined_df, 
        "Profitable Trades Overview", 
        "tab:profitable_trades_overview",
        ['Method', 'Frequency', 'Prof. Trades', 'Prof. Trades (TC)', 'Prof. Periods'],
        "Method & Freq. & Prof. Trades & Prof. Trades (TC) & Prof. Periods \\\\\n"
    )
    print(latex_table_1)
    print("---------------------")
    
    # Generate LaTeX table for Profit and Loss Trades
    latex_table_2 = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        "\\begin{tabular}{|p{3.5cm}|c|c|c|c|c|}\n"
        "\\hline\n"
        "Method & Freq. & $>0.6\\%$ & $>1\\%$ & $>5\\%$ & $>10\\%$ \\\\\n"
        "\\hline\n"
    )
    
    current_method = None
    for _, row in combined_df.iterrows():
        if row['Method'] != current_method:
            if current_method is not None:
                latex_table_2 += "\\hline\n"
            current_method = row['Method']
            latex_table_2 += f"\\multirow{{2}}{{*}}{{{row['Method']}}} & {row['Frequency']} & " + " & ".join([f"{row[col]:.2f}" if abs(row[col]) >= 1e-2 else "0.00" for col in ['Trades > 0.6%', 'Trades > 1%', 'Trades > 5%', 'Trades > 10%']]) + " \\\\\n"
        else:
            latex_table_2 += f" & {row['Frequency']} & " + " & ".join([f"{row[col]:.2f}" if abs(row[col]) >= 1e-2 else "0.00" for col in ['Trades > 0.6%', 'Trades > 1%', 'Trades > 5%', 'Trades > 10%']]) + " \\\\\n"
    
    latex_table_2 += (
        "\\hline\n"
        "\\end{tabular}\n"
        "\\caption{Profit and Loss Trades Overview}\n"
        "\\label{tab:profit_loss_trades_overview}\n"
        "\\end{table}"
    )
    print(latex_table_2)
    print("---------------------")
    
    # Generate LaTeX table for Return Metrics
    latex_table_3 = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        "\\begin{tabular}{|p{3.5cm}|c|c|c|c|c|}\n"
        "\\hline\n"
        "Method & Freq. & $\\mu$ Trade & $\\mu$ Period & $\\tilde{x}$ Trade & $\\tilde{x}$ Period \\\\\n"
        "\\hline\n"
    )
    
    current_method = None
    for _, row in combined_df.iterrows():
        if row['Method'] != current_method:
            if current_method is not None:
                latex_table_3 += "\\hline\n"
            current_method = row['Method']
            latex_table_3 += f"\\multirow{{2}}{{*}}{{{row['Method']}}} & {row['Frequency']} & " + " & ".join([f"{row[col]:.2f}" if abs(row[col]) >= 1e-2 else "0.00" for col in ['Avg. Trade', 'Avg. Period', 'Med. Trade', 'Med. Period']]) + " \\\\\n"
        else:
            latex_table_3 += f" & {row['Frequency']} & " + " & ".join([f"{row[col]:.2f}" if abs(row[col]) >= 1e-2 else "0.00" for col in ['Avg. Trade', 'Avg. Period', 'Med. Trade', 'Med. Period']]) + " \\\\\n"
    
    latex_table_3 += (
        "\\hline\n"
        "\\end{tabular}\n"
        "\\caption{Return Metrics Overview}\n"
        "\\label{tab:return_metrics_overview}\n"
        "\\end{table}"
    )
    print(latex_table_3)
    print("---------------------")

In [20]:
# Mapping of internal names to real names
labeling_method_names = {
    'excess_over_mean': 'Excess over Mean',
    'excess_over_median': 'Excess over Median',
    'fixed_time_horizon': 'Fixed Time Horizon',
    'triple_barrier': 'Triple Barrier',
    'tail_sets': 'Tail Sets',
    'matrix_flag': 'Matrix Flag',
    'trend_scanning': 'Trend Scanning',
    'buy_and_hold': 'Buy and Hold',
    'next_period': 'Next Period Labeling'
}

def rename_columns(df):
    """
    Rename columns for better clarity.
    """
    return df.rename(columns={
        'Labeling Method': 'Method',
        'Profitable Trades': 'Prof. Trades',
        'Profitable Trades after TC': 'Prof. Trades (TC)',
        'Percentage of Profitable Trades Total': 'Prof. Periods',
        'Percentage of Profitable Trades Total after TC': 'Prof. Periods (TC)',
        'Trades with >10% Profit': 'Trades > 10%',
        'Trades with >10% Loss': 'Trades < -10%',
        'Trades with >5% Profit': 'Trades > 5%',
        'Trades with >5% Loss': 'Trades < -5%',
        'Trades with >1% Profit': 'Trades > 1%',
        'Trades with >1% Loss': 'Trades < -1%',
        'Trades with >0.6% Profit': 'Trades > 0.6%',
        'Trades with >0.6% Loss': 'Trades < -0.6%',
        'Average Return per Trade': 'Avg. Trade',
        'Average Return per Period': 'Avg. Period',
        'Median Return per Trade': 'Med. Trade',
        'Median Return per Period': 'Med. Period'
    })

def map_labeling_methods(df):
    """
    Map internal labeling method names to real names.
    """
    df['Method'] = df['Method'].map(labeling_method_names)
    return df

def format_to_latex(df, title, label, columns, headers):
    """
    Format DataFrame to LaTeX table with merged cells.
    """
    latex_code = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        f"\\begin{{tabular}}{{|{'|'.join(['p{3.5cm}'] + ['c']*len(columns[2:]))}|}}\n"
        "\\hline\n"
        + headers +
        "\\hline\n"
    )
    
    current_method = None
    for _, row in df.iterrows():
        if row['Method'] != current_method:
            if current_method is not None:
                latex_code += "\\hline\n"
            current_method = row['Method']
            latex_code += f"\\multirow{{2}}{{*}}{{{row['Method']}}} & {row['Frequency']} & " + " & ".join([f"{row[col]:.3f}" if abs(row[col]) >= 1e-3 else "0.000" for col in columns[2:]]) + " \\\\\n"
        else:
            latex_code += f" & {row['Frequency']} & " + " & ".join([f"{row[col]:.3f}" if abs(row[col]) >= 1e-3 else "0.000" for col in columns[2:]]) + " \\\\\n"
    
    latex_code += (
        "\\hline\n"
        "\\end{tabular}\n"
        f"\\caption{{{title}}}\n"
        f"\\label{{{label}}}\n"
        "\\end{table}"
    )
    
    return latex_code

def print_trade_metrics_to_latex(daily_df, weekly_df):
    """
    This function takes two DataFrames (daily and weekly) with trade metrics and prints LaTeX tables comparing their metrics.
    
    Parameters:
    daily_df (DataFrame): The DataFrame containing the daily trade metrics.
    weekly_df (DataFrame): The DataFrame containing the weekly trade metrics.
    
    Returns:
    None: This function prints the LaTeX table codes.
    """
    # Rename columns
    daily_df = rename_columns(daily_df)
    weekly_df = rename_columns(weekly_df)
    
    # Map labeling methods
    daily_df = map_labeling_methods(daily_df)
    weekly_df = map_labeling_methods(weekly_df)
    
    # Combine daily and weekly DataFrames
    daily_df['Frequency'] = 'Daily'
    weekly_df['Frequency'] = 'Weekly'
    
    combined_df = pd.concat([daily_df, weekly_df], ignore_index=True)
    
    # Sort by method to ensure correct order
    combined_df.sort_values(by='Method', inplace=True)
    
    # Generate LaTeX table for Profitable Trades
    latex_table_1 = format_to_latex(
        combined_df, 
        "Profitable Trades Overview", 
        "tab:profitable_trades_overview",
        ['Method', 'Frequency', 'Prof. Trades', 'Prof. Trades (TC)', 'Prof. Periods'],
        "Method & Freq. & Prof. Trades & Prof. Trades (TC) & Prof. Periods \\\\\n"
    )
    print(latex_table_1)
    print("---------------------")
    
    # Generate LaTeX table for Profit and Loss Trades
    latex_table_2 = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        "\\begin{tabular}{|p{3.5cm}|c|c|c|c|c|}\n"
        "\\hline\n"
        "Method & Freq. & $>0.6\\%$ & $>1\\%$ & $>5\\%$ & $>10\\%$ \\\\\n"
        "\\hline\n"
    )
    
    current_method = None
    for _, row in combined_df.iterrows():
        if row['Method'] != current_method:
            if current_method is not None:
                latex_table_2 += "\\hline\n"
            current_method = row['Method']
            latex_table_2 += f"\\multirow{{2}}{{*}}{{{row['Method']}}} & {row['Frequency']} & " + " & ".join([f"{row[col]:.3f}" if abs(row[col]) >= 1e-3 else "0.000" for col in ['Trades > 0.6%', 'Trades > 1%', 'Trades > 5%', 'Trades > 10%']]) + " \\\\\n"
        else:
            latex_table_2 += f" & {row['Frequency']} & " + " & ".join([f"{row[col]:.3f}" if abs(row[col]) >= 1e-3 else "0.000" for col in ['Trades > 0.6%', 'Trades > 1%', 'Trades > 5%', 'Trades > 10%']]) + " \\\\\n"
    
    latex_table_2 += (
        "\\hline\n"
        "\\end{tabular}\n"
        "\\caption{Profit and Loss Trades Overview}\n"
        "\\label{tab:profit_loss_trades_overview}\n"
        "\\end{table}"
    )
    print(latex_table_2)
    print("---------------------")
    
    # Generate LaTeX table for Return Metrics
    latex_table_3 = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        "\\begin{tabular}{|p{3.5cm}|c|c|c|c|c|}\n"
        "\\hline\n"
        "Method & Freq. & $\\mu$ Trade & $\\mu$ Period & $\\tilde{x}$ Trade & $\\tilde{x}$ Period \\\\\n"
        "\\hline\n"
    )
    
    current_method = None
    for _, row in combined_df.iterrows():
        if row['Method'] != current_method:
            if current_method is not None:
                latex_table_3 += "\\hline\n"
            current_method = row['Method']
            latex_table_3 += f"\\multirow{{2}}{{*}}{{{row['Method']}}} & {row['Frequency']} & " + " & ".join([f"{row[col]:.3f}" if abs(row[col]) >= 1e-3 else "0.000" for col in ['Avg. Trade', 'Avg. Period', 'Med. Trade', 'Med. Period']]) + " \\\\\n"
        else:
            latex_table_3 += f" & {row['Frequency']} & " + " & ".join([f"{row[col]:.3f}" if abs(row[col]) >= 1e-3 else "0.000" for col in ['Avg. Trade', 'Avg. Period', 'Med. Trade', 'Med. Period']]) + " \\\\\n"
    
    latex_table_3 += (
        "\\hline\n"
        "\\end{tabular}\n"
        "\\caption{Return Metrics Overview}\n"
        "\\label{tab:return_metrics_overview}\n"
        "\\end{table}"
    )
    print(latex_table_3)
    print("---------------------")